In [1]:
import os
import sys
root_dir = os.path.abspath(os.path.join(os.path.dirname('../pruebillas.ipynb'), '..'))
os.chdir(root_dir)

sys.path.insert(0, os.path.join(root_dir, 'src'))

In [2]:
from backtesting import Backtest, Strategy
from backtesting.lib import crossover, plot_heatmaps, resample_apply, barssince
import pandas as pd
import talib as ta
import pandas_ta as pandas_ta
import numpy as np
import matplotlib.pyplot as plt
from backtesting import Strategy
import plotly.express as px

from datetime import datetime
import MetaTrader5 as mt5

import random
random.seed(42)

from backbone.utils.general_purpose import diff_pips

In [3]:
def plot_stats(data, stats, strategy, plot=False):
    equity_curve = stats._equity_curve
    aligned_data = data.reindex(equity_curve.index)
    bt = Backtest(aligned_data, strategy, cash=15_000, commission=0.002)
    print(stats)
    if plot:
        bt.plot(results=stats, resample=False)

In [4]:
def plot_full_equity_curve(data, stats_list, warmup_bars, lookback_bars, overlay_price=True):
    equity_curves = [x["_equity_curve"].iloc[warmup_bars:] for x in stats_list]

    combined = pd.Series(dtype=float)
    for curve in equity_curves:
        # normalized_curve = curve["Equity"] / curve["Equity"].iloc[0]  # Normaliza la curva a su valor inicial
        if combined.empty:
            combined = curve["Equity"]
        else:
            # Alinea la nueva curva con la última de la serie combinada
            # normalized_curve = normalized_curve * combined.iloc[-1]
            combined = pd.concat([combined, curve["Equity"]])

    fig = px.line(x=combined.index, y=combined)
    fig.update_traces(textposition="bottom right")
    fig.show()


In [5]:
symbols_path = './backbone/data/backtest/symbols/USDCAD.csv'
df = pd.read_csv(symbols_path)
df

,Date,Open,High,Low,Close,Volume
0,2021-08-02 00:00:00,1.24668,1.24721,1.24668,1.24719,118.27
1,2021-08-02 00:01:00,1.24722,1.24726,1.24709,1.24725,45.03
2,2021-08-02 00:02:00,1.24721,1.24721,1.24694,1.24713,87.82
3,2021-08-02 00:03:00,1.24714,1.24743,1.24713,1.24741,61.90
4,2021-08-02 00:04:00,1.24739,1.24739,1.24706,1.24720,94.39
...,...,...,...,...,...,...
897525,2024-01-01 23:55:00,1.32462,1.32462,1.32462,1.32462,2.40
897526,2024-01-01 23:56:00,1.32463,1.32465,1.32441,1.32443,43.27
897527,2024-01-01 23:57:00,1.32445,1.32450,1.32435,1.32436,23.85
897528,2024-01-01 23:58:00,1.32435,1.32436,1.32435,1.32435,9.60


In [6]:
train_start = '2021-08-01'
train_end = '2021-10-01'

test_start = '2022-01-01'
test_end = '2022-04-01'

train_data = df[(df.Date > train_start) & (df.Date < train_end)]
test_data = df[(df.Date > test_start) & (df.Date < test_end)]

train_data.loc[:, 'Date'] = pd.to_datetime(train_data.Date)
test_data.loc[:, 'Date'] = pd.to_datetime(test_data.Date)
df.loc[:, 'Date'] = pd.to_datetime(df.Date)

train_data = train_data.set_index('Date')
test_data = test_data.set_index('Date')

df = df.set_index('Date')


# Bullish engulfing Strategy

## Test Entry: Monkey Exit

In [14]:

class BullishEngulfing(Strategy):
    position_size = 3500
    pip_size = 0.0001
    sl_pips = 10
    rr = 2

    risk = 2

    def init(self):
        self.engulfing = self.I(ta.CDLENGULFING, self.data.Open, self.data.High, self.data.Low, self.data.Close)
        self.ema_200 = self.I(ta.EMA, self.data.Close, timeperiod=200)
        self.rsi = self.I(ta.RSI, self.data.Close, 14)

        # self.random = None

    def next(self):
        close_prices = self.data.Close
        actual_close_price = close_prices[-1]
       
        if self.position:
            pass
  
        else: 
            if actual_close_price > self.ema_200 and self.rsi > 55 and self.engulfing == 100:
                sl_price = actual_close_price - self.sl_pips * self.pip_size
                tp_price = actual_close_price + self.rr * self.sl_pips * self.pip_size

                account_risk = self.equity * (self.risk / 100)
                units = round(account_risk / (self.pip_size * self.sl_pips))
                self.buy(sl=sl_price, tp=tp_price, size=units)


            elif actual_close_price < self.ema_200 and self.rsi < 45 and self.engulfing == -100:
                sl_price = actual_close_price + self.sl_pips * self.pip_size
                tp_price = actual_close_price - self.rr * self.sl_pips * self.pip_size

                account_risk = self.equity * (self.risk / 100)
                units = round(account_risk / (self.pip_size * self.sl_pips))

                self.sell(sl=sl_price, tp=tp_price, size=units)

# metrics = pd.DataFrame()

# distances = [3, 4, 5]
# n_parameters = [5, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]
# parameter_combinations = list(itertools.product(
#     distances, n_parameters
# ))

# parameter_combinations

# for distance, n in parameter_combinations:
bt_train = Backtest(
    train_data, 
    BullishEngulfing, 
    commission=0.0002,
    cash=15_000, 
    margin=1/30
)

stats = bt_train.run()

stats

#     # bt_train.plot(filename='./RsiBBands.html', resample=False)

#     win_rate = stats['Win Rate [%]']
#     avg_trade_perc = stats['Avg. Trade [%]']

#     actual_metrics = pd.DataFrame(
#         {
#             'winning_rate':[win_rate], 
#             'avg_trade_perc':[avg_trade_perc], 
#             'n_candles':[n], 
#             'distance':[distance]
#         }
#     )

#     metrics = pd.concat([metrics, actual_metrics])

# metrics.winning_rate.median()

Start                     2021-08-02 00:00:00
End                       2021-09-30 23:59:00
Duration                     59 days 23:59:00
Exposure Time [%]                   73.882986
Equity Final [$]                  2358.526372
Equity Peak [$]                  15942.676715
Return [%]                         -84.276491
Buy & Hold Return [%]                1.700623
Return (Ann.) [%]                  -99.988925
Volatility (Ann.) [%]                0.025278
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -86.012747
Avg. Drawdown [%]                   -4.440407
Max. Drawdown Duration       59 days 05:11:00
Avg. Drawdown Duration        2 days 09:22:00
# Trades                                  392
Win Rate [%]                        34.693878
Best Trade [%]                       0.146744
Worst Trade [%]                     -0.105916
Avg. Trade [%]                    

In [ ]:
metrics.sort_values(by='winning_rate', ascending=False)

In [ ]:
bt_train.plot(filename='./RsiBBands.html', resample=False)


## Test Exit: Monkey Entry

In [ ]:

trade_frecuency = stats['# Trades'] / train_data.shape[0]
long_frecuency = stats._trades[stats._trades['Size'] > 0].shape[0] / stats['# Trades']
short_frecuency = stats._trades[stats._trades['Size'] < 0].shape[0] / stats['# Trades']

def random_boolean(prob_true=0.5):
    return random.choices([True, False], weights=[prob_true, 1 - prob_true], k=1)[0]


class VwapExit(Strategy):
    pip_size = 0.0001
    sl_pips = 15
    rr = 1.5
    risk = 1

    adx_period = 5
    rsi_period = 3

    def init(self):
        self.rsi = self.I(ta.RSI, self.data.Close, 14)

        self.random = None

    def next(self):
        close_prices = self.data.Close
        actual_close = close_prices[-1]
       
        if self.position:
            if self.position.is_long and self.rsi > 70:
                self.position.close()
                
            elif self.position.is_short and self.rsi < 30:
                self.position.close()

        else: 
            trade = random_boolean(prob_true=trade_frecuency)

            if trade:
                long = random_boolean(prob_true=long_frecuency)

                if long:
                    sl = actual_close - self.sl_pips * self.pip_size
                    account_risk = self.equity * (self.risk / 100)
                    units = round(account_risk / (self.pip_size * self.sl_pips))
                    self.buy(sl=sl, size=units)

                else:
                    sl = actual_close + self.sl_pips * self.pip_size
                    account_risk = self.equity * (self.risk / 100)
                    
                    # Calculate lot size in units
                    units = round(account_risk / (self.pip_size * self.sl_pips))
                    self.sell(sl=sl, size=units)


metrics = pd.DataFrame()

for x in range(0, 10):
    bt_train = Backtest(
        train_data, 
        VwapExit, 
        cash=15_000, 
        margin=1/30
    )

    stats = bt_train.run(
    )

    # bt_train.plot(filename='./RsiBBands.html', resample=False)

    equity = stats['Equity Final [$]']
    return_ = stats['Return [%]']
    sharpe_ratio = stats['Sharpe Ratio']

    actual_metrics = pd.DataFrame(
        {
            'equity':[equity], 
            'return_':[return_], 
            'sharpe_ratio':[sharpe_ratio], 
        }
    )

    metrics = pd.concat([metrics, actual_metrics])



In [ ]:
metrics.return_.std()

## Full system

In [7]:
import itertools

class VwapRsiFull(Strategy):
    pip_size = 0.0001
    sl_pips = 10
    rr = 1
    risk = 1

    n_candles = 7 
    distance = 1

    rsi_up_threshold=70
    rsi_down_threshold=30


    def init(self):
        self.vwap = self.I(
            pandas_ta.vwap, 
            pd.Series(self.data.High, index=self.data.index),
            pd.Series(self.data.Low, index=self.data.index),
            pd.Series(self.data.Close, index=self.data.index),
            pd.Series(self.data.Volume, index=self.data.index),
        ) 

        self.ema_50 = self.I(ta.EMA, self.data.Close, timeperiod=200) 
        
        self.rsi = self.I(ta.RSI, self.data.Close, 14)


    def next(self):
        actual_close = self.data.Close[-1]
       
        if self.position:
            pass

        else: 

            actual_vwap = self.vwap[-1]
            distance_vwap = diff_pips(actual_vwap, actual_close, pip_value=self.pip_size, absolute=True)

            n_candles_under_vwap = True
            n_candles_up_vwap = True

            for x in range(1, self.n_candles):
                if self.data.Close[-x] > self.vwap[-x]:
                    n_candles_under_vwap = False

                if self.data.Close[-x] < self.vwap[-x]:
                    n_candles_up_vwap = False

            if distance_vwap <= self.distance and n_candles_up_vwap: 
                sl = actual_close - self.sl_pips * self.pip_size
                tp = actual_close + self.rr * self.sl_pips * self.pip_size
                
                account_risk = self.equity * (self.risk / 100)
                units = round(account_risk / (self.pip_size * self.sl_pips))
                self.buy(sl=sl, size=units, tp=tp)

            elif distance_vwap <= self.distance and n_candles_under_vwap: 
                sl = actual_close + self.sl_pips * self.pip_size
                tp = actual_close - self.rr * self.sl_pips * self.pip_size

                account_risk = self.equity * (self.risk / 100)
                
                # Calculate lot size in units
                units = round(account_risk / (self.pip_size * self.sl_pips))
                self.sell(sl=sl, size=units, tp=tp)


bt_train = Backtest(
    train_data, 
    VwapRsiFull, 
    commission=0.0002,
    cash=15_000, 
    margin=1/30
)


stats = bt_train.run(rr=2, sl_pips=15)

# stats = bt_train.optimize(
#     distance=[1, 2, 3],
#     n_candles=[6, 8,  10, 12],
#     sl_pips=[5, 8, 12, 15],
#     rsi_up_threshold=[60, 70, 80],
#     rsi_down_threshold=[20, 30, 40],
# )

bt_train.plot(filename='./RsiBBands.html', resample=False)

stats


c:\Users\SaidJ\OneDrive\Documentos\projects\forex_ml_bot\forex_ml_bot\mtvenv\lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
c:\Users\SaidJ\OneDrive\Documentos\projects\forex_ml_bot\forex_ml_bot\mtvenv\lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],


Start                     2021-08-02 00:00:00
End                       2021-09-30 23:59:00
Duration                     59 days 23:59:00
Exposure Time [%]                   47.719562
Equity Final [$]                 15663.244493
Equity Peak [$]                  16767.685706
Return [%]                            4.42163
Buy & Hold Return [%]                1.700623
Return (Ann.) [%]                   38.270477
Volatility (Ann.) [%]               40.637479
Sharpe Ratio                         0.941753
Sortino Ratio                        1.967746
Calmar Ratio                         2.676103
Max. Drawdown [%]                  -14.300821
Avg. Drawdown [%]                   -0.765879
Max. Drawdown Duration       34 days 22:51:00
Avg. Drawdown Duration        0 days 16:24:00
# Trades                                   98
Win Rate [%]                        40.816327
Best Trade [%]                       0.235055
Worst Trade [%]                     -0.160246
Avg. Trade [%]                    

In [ ]:
stats._strategy.__dict__

# WFO

In [ ]:
def walk_forward(
        strategy,
        data_full,
        warmup_bars,
        lookback_bars=28*1440,
        validation_bars=7*1440,
        cash=15_000, 
        commission=0.0002,
        margin=1/30

):

    stats_master = []
    equity_final = None

    for i in range(lookback_bars, len(data_full)-validation_bars, validation_bars):

        # print(i)

        # To do anchored walk-forward, just set the first slice here to 0
        train_data = data_full.iloc[i-lookback_bars: i]

        print(f'train from {train_data.index[0]} to {train_data.index[-1]}')

        bt_training = Backtest(
            train_data, 
            strategy, 
            cash=cash, 
            commission=commission, 
            margin=margin
        )

        stats_training = bt_training.optimize(
            sl_pips=[15],
            n_candles=[30, 60, 90, 120],
            rr=[1, 1.5, 2, 3],
            maximize='Return [%]'
        )
        
        validation_data = data_full.iloc[i-warmup_bars:i+validation_bars]
        print(f'validate from {validation_data.index[0]} to {validation_data.index[-1]}')


        bt_validation = Backtest(
            validation_data, 
            strategy, 
            cash=cash if equity_final == None else equity_final, 
            commission=commission, 
            margin=margin
        )

        stats_validation = bt_validation.run(
                sl_pips = stats_training._strategy.sl_pips,
                rr = stats_training._strategy.rr,
                n_candles = stats_training._strategy.n_candles,
        )
        
        equity_final = stats_validation['Equity Final [$]']
        print(f'equity final: {equity_final}')

        stats_master.append(stats_validation)

    return stats_master

In [ ]:
import pickle


lookback_bars = 60*1440
validation_bars = 7*1440
warmup_bars = 14*60

if os.path.exists("stats.pickle"):
    with open("stats.pickle", "rb") as f:
        stats = pickle.load(f)
else:
    stats = walk_forward(
        VwapRsiFull,
        test_data, 
        warmup_bars=0, 
        commission=0, 
        margin=1/30, 
        cash=15_000
    )
    
    # with open("stats.pickle", "wb") as f:
    #     pickle.dump(stats, f)

In [ ]:
stats = pd.DataFrame(stats)
stats['Win Rate [%]'].mean()

In [ ]:
plot_stats(df, stats[3], VwapRsiFull, plot=True)

In [ ]:
plot_full_equity_curve(df, stats, warmup_bars = warmup_bars,
                       lookback_bars = lookback_bars, overlay_price = True)

In [ ]:
trades = test_stats._trades.groupby(by=['ExitTime']).agg({'PnL':['sum','count'], 'Duration':'max'})
trades.columns = trades.columns.droplevel(0)
trades = trades.reset_index().rename(columns={'count':'ammount_trades'})
trades = trades.rename(columns={'sum':'profit'})
trades = trades.rename(columns={'max':'minutes_in_trade'})
trades

In [ ]:
trades.minutes_in_trade.describe()
